In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.svm import LinearSVC
import numpy as np
from linear_models import LinearOneVsAllClassifier
from utils import generate_exp_data
from mwu import run_mwu
from noise_functions_multi import grad_desc_nonconvex
import ray
import os

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
mnist_images = np.copy(mnist.train.images)
mnist_labels = np.argmax(mnist.train.labels, axis=1)

In [4]:
mnist_test_images = mnist.test.images
mnist_test_labels = np.argmax(mnist.test.labels, axis=1)

In [5]:
num_mnist_features = 784

In [6]:
num_models = 10
linear_models = []
zeroed_features_list = []
sparse_training_sets = []
for i in xrange(num_models):
    sparse_mnist_images = np.copy(mnist_images)
    zeroed_features = np.random.randint(0, num_mnist_features, 588)
    zeroed_features_list.append(zeroed_features)
    sparse_mnist_images[:, zeroed_features] = 0.0
    sparse_training_sets.append(sparse_mnist_images)

In [7]:
@ray.remote
def train_model(sparse_mnist_images, mnist_labels):
    model = LinearSVC(loss='hinge')
    model.fit(sparse_mnist_images, mnist_labels)
    return LinearOneVsAllClassifier(10, model.coef_, model.intercept_) 

In [8]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:56888 to respond...
Waiting for redis server at 127.0.0.1:12825 to respond...
Starting local scheduler with the following resources: {'GPU': 0, 'CPU': 4}.

View the web UI at http://localhost:8906/notebooks/ray_ui22025.ipynb?token=69ca904c63a333114ed121346e71d9a386d1c3d409e9aa5d



{'local_scheduler_socket_names': ['/tmp/scheduler16710789'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store41359419', manager_name='/tmp/plasma_manager77173156', manager_port=12238)],
 'redis_address': '127.0.0.1:56888',
 'webui_url': 'http://localhost:8906/notebooks/ray_ui22025.ipynb?token=69ca904c63a333114ed121346e71d9a386d1c3d409e9aa5d'}

In [9]:
models = [train_model.remote(train_set, mnist_labels) for train_set in sparse_training_sets]
models = ray.get(models)

In [10]:
exp_folder = 'generalization_experiment'
os.mkdir(exp_folder)
os.mkdir(exp_folder + '/models/')
os.mkdir(exp_folder + '/results')
os.mkdir(exp_folder + '/data/')

for i, model in enumerate(models):
    np.save('{}/models/w_{}.npy'.format(exp_folder, i), model.weights)
    np.save('{}/models/b_{}.npy'.format(exp_folder, i), model.bias)

In [11]:
num_points = 1000
X_exp, Y_exp = generate_exp_data(num_points, mnist_test_images, mnist_test_labels, models)

In [12]:
np.save(exp_folder  + '/data/X_exp.npy', X_exp)
np.save(exp_folder  + '/data/Y_exp.npy', X_exp)

In [19]:
# subset_sizes = [5, 10, 25, 50, 100, 150, 250, 500, 750, 1000]
subset_sizes = [2, 3]
mwu_iters = 1
alpha = .5
for k in subset_sizes:
    chosen_ixs = np.random.choice(range(num_models), k)
    chosen_models = []
    for ix in chosen_ixs:
        chosen_models.append(models[ix])
    print chosen_ixs
#     weights, noise, loss_history, acc_history, action_loss = run_mwu(chosen_models, mwu_iters, X_exp, Y_exp, alpha,
#                                                                      grad_desc_nonconvex)
#     np.save(exp_folder + "/results/" + "weights_{}.npy".format(k), weights)
#     np.save(exp_folder + "/results/" + "noise_{}.npy".format(k), noise)
#     np.save(exp_folder + "/results/" + "loss_history_{}.npy".format(k), loss_history)
#     np.save(exp_folder + "/results/" + "acc_history_{}.npy".format(k), acc_history)
#     np.save(exp_folder + "/results/" + "action_loss_{}.npy".format(k), action_loss)
print "DONE"

[2 8]
[5 8 9]
DONE
